In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.datasets import mnist

Using TensorFlow backend.


In [2]:
np.random.seed(42)

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
X_train.shape

(60000, 28, 28)

In [5]:
X_train = X_train.reshape(60000, 784)
X_train = X_train.astype('float32')
X_train /= 255

In [6]:
y_train = np_utils.to_categorical(y_train, 10)

In [7]:
y_train[0]

array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.])

## Создаём последовательную модель

In [8]:
model = Sequential()
#Добавляем уровни сети
model.add(Dense(800, input_dim=784, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, kernel_initializer='normal', activation='softmax'))

In [9]:
#Компилируем модель
model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 800)               628000    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                8010      
Total params: 636,010
Trainable params: 636,010
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
%%time
#Обучаем сеть
model.fit(X_train, y_train, batch_size=1000, epochs=5, verbose=True)

Epoch 1/5
60000/60000 [==============================] - 5s - loss: 1.9499 - acc: 0.4179      

In [11]:
# Запускаем сеть на входных данных
predictions = model.predict(X_train)

In [12]:
# Преобразуем выходные данные сети из категорий в метки классов
predictions = np.argmax(predictions, axis = 1)

In [13]:
predictions

array([5, 0, 4, ..., 5, 6, 8])

In [14]:
# Оцениваем качество обучения сети на тестовых данных
scores = model.evaluate(X_test.reshape(10000, 784), np_utils.to_categorical(y_test), verbose=True)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1] * 100))

 9600/10000 [===========================>..] - ETA: 0s 

## Сохраняем обученную модель

In [15]:
# Генерируем описание модели в формате json
model_json = model.to_json()

In [16]:
# Записываем модель в файл
with open("mnist_model_simple.json", 'w') as json_file: 
    json_file.write(model_json)

In [17]:
!cat mnist_model_simple.json

{"class_name": "Sequential", "config": [{"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "batch_input_shape": [null, 784], "dtype": "float32", "units": 800, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "RandomNormal", "config": {"mean": 0.0, "stddev": 0.05, "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_2", "trainable": true, "units": 10, "activation": "softmax", "use_bias": true, "kernel_initializer": {"class_name": "RandomNormal", "config": {"mean": 0.0, "stddev": 0.05, "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}], "keras_version": "2.0.5", "backend": 

In [ ]:
# Сохраняем веса модели
model.save_weights("mnist_model_simple_weights.h5")

## Загружаем готовую модель

In [20]:
#Загружаем данные об архитектуре сети из файла json
with open('mnist_model_simple.json', 'r') as json_file:
    loaded_model_json = json_file.read()

In [21]:
#Создаём модель на основе загруженных данных
from keras.models import model_from_json
loaded_model = model_from_json(loaded_model_json)

In [22]:
# Загружаем веса в модель
loaded_model.load_weights('mnist_model_simple_weights.h5')

In [24]:
# Компилируем модель
loaded_model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

In [25]:
# Проверим, что качество модели не поменялось
scores = model.evaluate(X_test.reshape(10000, 784), np_utils.to_categorical(y_test), verbose=True)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1] * 100))

 9792/10000 [============================>.] - ETA: 0s Точность работы на тестовых данных: 85.40%
